In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [2]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
url = "https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1"
driver.get(url)
driver.implicitly_wait(2)

In [3]:
def get_additional_info(xpath_src):
    """
    코디맵 클릭 후 다시 이전 페이지로 돌아오는 함수
    """
    info = []
    item_btn = driver.find_element(By.XPATH, xpath_src)
    item_btn.click()
    
    soup2 = BeautifulSoup(driver.page_source, 'lxml')
    
    style_info = soup2.find('div', attrs={'id': 'style_info'})
    
    # codimap_explain
    explain = style_info.find('p', attrs={'class': 'styling_txt'})
    explain = explain.get_text().strip() if explain else ''
    info.append(explain)
    
    # codimap_hashtag
    hashtag_list = style_info.find_all('a', attrs={'class': 'ui-tag-list__item'})
    hashtag = ''
    for tag in hashtag_list:
        tag = tag.get_text().strip() if tag else ''
        hashtag += tag + ' '
    info.append(hashtag[:-1])
    
    # codimap_imgurl
    imgurl = style_info.find('img', attrs={'class': 'photo'})
    imgurl = imgurl['src'] if imgurl['src'] else ''
    info.append(imgurl)
    
    driver.back()
    
    return info

In [4]:
columns = [
    'codimap_category',
    'codimap_title',
    'codimap_date',
    'views',
    'comment_numbers'
]
columns_ext = [
    'codimap_explain',
    'codimap_hashtag',
    'codimap_imgurl'
]
total_items = []

for page in range(5):
    url = f'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={page+1}'
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    list_items = soup.find_all('li', attrs={'class': 'style-list-item'})
    for i, li in enumerate(list_items):
        print(f'{page+1}페이지: {i+1}번째 코디맵 크롤링')
        item = []

        # codimap_category
        category = li.find('span', attrs={'class': 'style-list-information__text'})
        category = category.get_text().strip() if category else 'Something is wrong'
        item.append(category)

        # codimap_title
        title = li.find('strong', attrs={'class': 'style-list-information__title'})
        title = title.get_text().strip() if title else 'Something is wrong'
        item.append(title)

        post_info = li.find_all('span', attrs={'class': 'post-information__text'})
        date, views, comment_cnt = post_info[0], post_info[1], post_info[2]

        # codimap_date
        date = date.get_text().strip() if date else 'Something is wrong'
        item.append(date)

        # views
        views = views.get_text().strip().split()[1].replace(',', '')
        item.append(int(views))

        # comment_numbers
        comment_cnt = comment_cnt.get_text().strip().split()[1]
        comment_cnt.replace(',', '')
        item.append(int(comment_cnt))

        xpath = f'/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{i+1}]/div[1]/a'
        additional_info = get_additional_info(xpath)
        item += additional_info

        total_items.append(item)
        print('---------------------------------------------------')

columns += columns_ext
df = pd.DataFrame(total_items, columns=columns)

1페이지: 1번째 코디맵 크롤링
---------------------------------------------------
1페이지: 2번째 코디맵 크롤링
---------------------------------------------------
1페이지: 3번째 코디맵 크롤링
---------------------------------------------------
1페이지: 4번째 코디맵 크롤링
---------------------------------------------------
1페이지: 5번째 코디맵 크롤링
---------------------------------------------------
1페이지: 6번째 코디맵 크롤링
---------------------------------------------------
1페이지: 7번째 코디맵 크롤링
---------------------------------------------------
1페이지: 8번째 코디맵 크롤링
---------------------------------------------------
1페이지: 9번째 코디맵 크롤링
---------------------------------------------------
1페이지: 10번째 코디맵 크롤링
---------------------------------------------------
1페이지: 11번째 코디맵 크롤링
---------------------------------------------------
1페이지: 12번째 코디맵 크롤링
---------------------------------------------------
1페이지: 13번째 코디맵 크롤링
---------------------------------------------------
1페이지: 14번째 코디맵 크롤링
---------------------------------------------------
1페이지: 15번째 코디맵 

---------------------------------------------------
2페이지: 58번째 코디맵 크롤링
---------------------------------------------------
2페이지: 59번째 코디맵 크롤링
---------------------------------------------------
2페이지: 60번째 코디맵 크롤링
---------------------------------------------------
3페이지: 1번째 코디맵 크롤링
---------------------------------------------------
3페이지: 2번째 코디맵 크롤링
---------------------------------------------------
3페이지: 3번째 코디맵 크롤링
---------------------------------------------------
3페이지: 4번째 코디맵 크롤링
---------------------------------------------------
3페이지: 5번째 코디맵 크롤링
---------------------------------------------------
3페이지: 6번째 코디맵 크롤링
---------------------------------------------------
3페이지: 7번째 코디맵 크롤링
---------------------------------------------------
3페이지: 8번째 코디맵 크롤링
---------------------------------------------------
3페이지: 9번째 코디맵 크롤링
---------------------------------------------------
3페이지: 10번째 코디맵 크롤링
---------------------------------------------------
3페이지: 11번째 코디맵 크롤링
---------------

---------------------------------------------------
4페이지: 54번째 코디맵 크롤링
---------------------------------------------------
4페이지: 55번째 코디맵 크롤링
---------------------------------------------------
4페이지: 56번째 코디맵 크롤링
---------------------------------------------------
4페이지: 57번째 코디맵 크롤링
---------------------------------------------------
4페이지: 58번째 코디맵 크롤링
---------------------------------------------------
4페이지: 59번째 코디맵 크롤링
---------------------------------------------------
4페이지: 60번째 코디맵 크롤링
---------------------------------------------------
5페이지: 1번째 코디맵 크롤링
---------------------------------------------------
5페이지: 2번째 코디맵 크롤링
---------------------------------------------------
5페이지: 3번째 코디맵 크롤링
---------------------------------------------------
5페이지: 4번째 코디맵 크롤링
---------------------------------------------------
5페이지: 5번째 코디맵 크롤링
---------------------------------------------------
5페이지: 6번째 코디맵 크롤링
---------------------------------------------------
5페이지: 7번째 코디맵 크롤링
-------------

In [6]:
df.tail()

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
295,스트릿,힙하게 입어봐,22.10.11,3771,3,자수 디테일이 돋보이는 스타디움 재킷과 데님 팬츠를 매치하고 스니커즈로 멋을 더한 ...,#모노톤 #유니크 #힙합 #심볼 #그래픽 #벨루어 #자수 #컬러블록 #가을 #스트릿...,//image.msscdn.net/images/codimap/detail/19131...
296,캐주얼,미니멀의 정석,21.01.04,1786,3,담백한 디테일의 롱 코트에 니트와 팬츠를 매치하고 로고 패턴이 돋보이는 스니커즈로 ...,#겨울 #캐주얼 #엠씨엠 #아모멘토 #캔버스/단화 #니트/스웨터 #셔츠/블라우스 #...,//image.msscdn.net/images/codimap/detail/1263/...
297,캐주얼,블루 활용법,21.01.04,9079,3,화사한 블루 컬러 숏 패딩에 스웨트셔츠와 카고 팬츠를 매치하고 스니커즈로 마무리한 ...,#겨울 #캐주얼 #어나더유스 #트래블 #리끌로우 #앳 더 모먼트 #커버낫 #반스 #...,//image.msscdn.net/images/codimap/detail/1276/...
298,걸리시,퍼 재킷 연출하기,21.01.05,1839,3,체크 패턴이 돋보이는 미니스커트와 퍼 재킷을 매치하고 부츠로 완성한 걸리시 룩,#겨울 #걸리시 #시티브리즈 #조셉앤스테이시 #커렌트 #어글리쉐도우 #에이티티 #미...,//image.msscdn.net/images/codimap/detail/1282/...
299,캐주얼,편하게 꾸며,21.06.28,4158,3,프린팅 티셔츠와 데님 팬츠를 매치하고 메신저 백으로 포인트를 준 캐주얼 룩,#캐릭터 #그래픽 #와이드핏 #꾸안꾸 #캠퍼스 #여름 #캐주얼 #모 스포츠 #피지컬...,//image.msscdn.net/images/codimap/detail/3939/...


In [7]:
df.to_csv("무신사_크롤링.csv", encoding='utf-8')